In [ ]:

import xarray as xr
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
from xmovie import Movie
import imageio
import os
from dask.distributed import Client, LocalCluster
from datetime import datetime,timedelta
import glob
import indices_function as ifun
import warnings
from rechunker import rechunk

warnings.filterwarnings('ignore')



In [ ]:

cluster = LocalCluster(
    n_workers=20, 
    threads_per_worker=1,
    timeout='5600s',
    memory_limit='200GB',   
)
client = Client(cluster)
client


In [ ]:
import dask
import xarray as xr

# List of file names
models=glob.glob("/nobackupp10/datapool/NEX-DCP30-CMIP6/*")
for model in models[:1]:
    file_names=glob.glob(model+"/historical/*/pr/*.nc")
    file_names.sort()
    datasets =xr.open_mfdataset(file_names[34:]).chunk({"time":-1,"lat":700,"lon":700})
    # datasets=datasets.where(datasets.pr<=0.256,0,datasets.pr)
    quantile_models = ifun.nonzero_quantiles(datasets,[0.25,0.5,0.75,0.9])
    target_chunks = {
        "pr": {"quantile": 2, "lat": 700, "lon": 700},
        "quantile": None,  # don't rechunk this array
        "lon": None,
        "lat": None,
    }
    max_mem ="100GB"
    os.system("rm -rf /nobackupp10/skhajehe/dcp-indices/multimodel/quantile/temp_chunk_pr.zarr")
    os.system("rm -rf /nobackupp10/skhajehe/dcp-indices/multimodel/quantile/pr/"+model.split("/")[-1]+"_test.zarr")
    target_store = "/nobackupp10/skhajehe/dcp-indices/multimodel/quantile/pr/"+model.split("/")[-1]+"_test.zarr"
    temp_store = "/nobackupp10/skhajehe/dcp-indices/multimodel/quantile/temp_chunk_pr.zarr"
    # need to remove the existing stores or it won't work
    # !rm -rf group_rechunked.zarr group_rechunked-tmp.zarr
    array_plan = rechunk(
        quantile_models, target_chunks, max_mem, target_store, temp_store=temp_store
    )
    array_plan.execute()


In [ ]:
import dask
import xarray as xr

variables=["vpr"]
for var in variables:
    file_names=glob.glob("/nobackupp10/nexprojects/GDDP2/obs/prism/day/"+var+"/*.nc")
    file_names.sort()
    # Open each file lazily using Dask and xarray
    datasets =xr.open_mfdataset(file_names[3:]).chunk({"time":-1,"lat":700,"lon":700})
    # Concatenate datasets along the time dimension using Dask and xarray
    quantile_models = ifun.nonzero_quantiles(datasets,[0.25,0.5,0.75,0.9])

    target_chunks = {
        var: {"quantile": 4, "lat": 700, "lon": 700},
        "quantile": None,  # don't rechunk this array
        "lon": None,
        "lat": None,
    }
    max_mem ="100GB"
    os.system("rm -rf /nobackupp28/skhajehe/dcp-indices/prism/quantile/temp_chunk.zarr")
    os.system("rm -rf /nobackupp28/skhajehe/dcp-indices/prism/quantile/"+var+"/quantiles_sort.zarr")
    target_store = "/nobackupp28/skhajehe/dcp-indices/prism/quantile/"+var+"/quantiles_sort.zarr"
    temp_store = "/nobackupp28/skhajehe/dcp-indices/prism/quantile/temp_chunk.zarr"
    # need to remove the existing stores or it won't work
    # !rm -rf group_rechunked.zarr group_rechunked-tmp.zarr
    array_plan = rechunk(
        quantile_models, target_chunks, max_mem, target_store, temp_store=temp_store
    )
    array_plan.execute()
